### Three approaches:
## 1.Drop Categorical Variables
## 2.Label Encoding
## 3.One-Hot Encoding

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# read the data
melb = pd.read_csv('../../data/melb_data.csv')

In [50]:
y = melb.Price
X = melb.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3)

In [51]:
# find columns with missing values
miss = [col for col in X_train_full.columns if X_train_full[col].isnull().any()]

In [52]:
# drop columns with missing values
X_train_full.drop(miss, axis=1, inplace=True)
X_valid_full.drop(miss, axis=1, inplace=True)

/home/dddiiirrr3/PycharmProjects/mlearning/venv/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [53]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]

In [54]:
# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [55]:
# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols

In [56]:
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [57]:
X_train.head(10)


,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
4057,h,SP,Western Metropolitan,4,6.9,3039.0,4.0,1.0,388.0,-37.76920,144.90570,6232.0
3936,u,S,Western Metropolitan,2,8.7,3032.0,2.0,2.0,0.0,-37.77520,144.89250,4918.0
3099,h,S,Southern Metropolitan,3,14.5,3188.0,3.0,2.0,614.0,-37.93460,145.03020,2356.0
12569,h,S,Western Metropolitan,4,6.2,3015.0,4.0,2.0,409.0,-37.84445,144.87966,5498.0
3545,h,S,Northern Metropolitan,3,4.2,3031.0,3.0,1.0,0.0,-37.79570,144.93200,5263.0
13136,h,VB,Eastern Metropolitan,3,14.7,3151.0,3.0,1.0,189.0,-37.85777,145.14009,4048.0
10254,u,SP,Northern Metropolitan,3,19.6,3076.0,3.0,2.0,156.0,-37.63490,145.04042,10926.0
9929,h,S,Northern Metropolitan,3,14.0,3047.0,3.0,1.0,627.0,-37.68861,144.91065,851.0
7555,h,S,Southern Metropolitan,3,11.2,3186.0,3.0,2.0,393.0,-37.90200,144.99220,10579.0
5392,h,S,Northern Metropolitan,2,2.6,3121.0,2.0,1.0,158.0,-37.82010,145.00700,14949.0


In [58]:
# get the list of categorical variables
s = (X_train.dtypes == 'object')

In [59]:
object_cols = list(s[s].index)

In [60]:
print("Categorical variables:")
print(object_cols)

Categorical variables:
['Type', 'Method', 'Regionname']


In [69]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [62]:
# 1 approach

drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
187811.0926795172


In [27]:
X_train

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.98670,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.90050,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.82200,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.91580,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.82720,4217.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13123,h,SP,Northern Metropolitan,3,5.2,3056.0,3.0,1.0,212.0,-37.77695,144.95785,11918.0
3264,h,S,Eastern Metropolitan,3,10.5,3081.0,3.0,1.0,748.0,-37.74160,145.04810,2947.0
9845,h,PI,Northern Metropolitan,4,6.7,3058.0,4.0,2.0,441.0,-37.73572,144.97256,11204.0
10799,h,S,Northern Metropolitan,3,12.0,3073.0,3.0,1.0,606.0,-37.72057,145.02615,21650.0


In [63]:
# 2 approach

from sklearn.preprocessing import LabelEncoder

# make copy to avoid changing original data
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()

for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])

print("MAE from Approach 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))  

MAE from Approach 2 (Label Encoding):
176427.71771162073


In [64]:
# 3 approach

from sklearn.preprocessing import OneHotEncoder

# apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# one-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
174798.2112062557


Index([        'Rooms',      'Distance',      'Postcode',      'Bedroom2',
            'Bathroom',      'Landsize',     'Lattitude',    'Longtitude',
       'Propertycount',               0,               1,               2,
                     3,               4,               5,               6,
                     7,               8,               9,              10,
                    11,              12,              13,              14,
                    15],
      dtype='object')

In [67]:
df_train = pd.read_csv('../../data/home-data-for-ml-course/train.csv', index_col='Id')
df_test = pd.read_csv('../../data/home-data-for-ml-course/test.csv', index_col='Id')

# remove rows with missing target, separate target from predictots
df_train.dropna(axis=0, subset=['SalePrice'], inplace=True)
df_train_y = df_train.SalePrice
df_train.drop(['SalePrice'], axis=1, inplace=True)

# to keep things simple, we'll drop columns with missing values
missing = [col for col in df_train.columns if df_train[col].isnull().any()]
df_train.drop(missing, axis=1, inplace=True)
df_test.drop(missing, axis=1, inplace=True)

# break off validation set from training data
X_df_train, X_df_valid, y_df_train, y_df_valid = train_test_split(df_train, df_train_y, train_size=0.8, test_size=0.2, random_state=0)


In [68]:
X_df_train.shape

(1168, 60)

In [70]:
# approach 1
# drop columns in training and validation data
drop_X_df_train = X_df_train.select_dtypes(exclude=['object'])
drop_X_df_valid = X_df_valid.select_dtypes(exclude=['object'])


In [71]:
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_df_train, drop_X_df_valid, y_df_train, y_df_valid))

MAE from Approach 1 (Drop categorical variables):
17837.82570776256


In [72]:
# approach 2
print("Unique values in 'Condition2' column in training data:", X_df_train['Condition2'].unique())
print("\nUnique values in 'Condition2' column in validation data:", X_df_valid['Condition2'].unique())

Unique values in 'Condition2' column in training data: ['Norm' 'PosA' 'Feedr' 'PosN' 'Artery' 'RRAe']

Unique values in 'Condition2' column in validation data: ['Norm' 'RRAn' 'RRNn' 'Artery' 'Feedr' 'PosN']


In [74]:
# all categorical columns
obj_cols = [col for col in X_df_train.columns if X_df_train[col].dtype == 'object']

In [76]:
# columns that can be safely label encoded
good_label_cols = [col for col in obj_cols if set(X_df_train[col]) == set(X_df_valid[col])]

In [78]:
# problematic columns that will be dropped from the dataset
bad_label_cols = list(set(obj_cols) - set(good_label_cols))

In [79]:
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be label encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'BldgType', 'HouseStyle', 'ExterQual', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Foundation', 'Exterior2nd', 'Utilities', 'LandSlope', 'SaleType', 'Exterior1st', 'RoofMatl', 'ExterCond', 'HeatingQC', 'Condition2', 'RoofStyle', 'Heating', 'Functional', 'Condition1', 'Neighborhood']


In [83]:
# drop categorical columns that will not be encoded
label_X_df_train = X_df_train.drop(bad_label_cols, axis=1)
label_X_df_valid = X_df_valid.drop(bad_label_cols, axis=1)

# apply label encoder
coder = LabelEncoder()

for col in good_label_cols:
    label_X_df_train[col] = coder.fit_transform(X_df_train[col])
    label_X_df_valid[col] = coder.transform(X_df_valid[col])

In [84]:
print("MAE from Approach 2 (Label Encoding):") 
print(score_dataset(label_X_df_train, label_X_df_valid, y_df_train, y_df_valid))

MAE from Approach 2 (Label Encoding):
17575.291883561644
